In [7]:
# MODI Method via NWCR

import numpy as np

def get_balanced_tp(supply, demand, costs, penalties = None):
    total_supply = sum(supply)
    total_demand = sum(demand)
    
    if total_supply < total_demand:
        if penalties is None:
            raise Exception('Supply less than demand, penalties required')
        new_supply = supply + [total_demand - total_supply]
        new_costs = costs + [penalties]
        return new_supply, demand, new_costs
    if total_supply > total_demand:
        new_demand = demand + [total_supply - total_demand]
        new_costs = costs + [[0 for _ in demand]]
        return supply, new_demand, new_costs
    return supply, demand, costs

def north_west_corner(supply, demand,costs):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    i = 0
    j = 0
    sum1=0
    bfs = []
    while len(bfs) < len(supply) + len(demand) - 1:
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s, d)
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i, j), v))
        sum1=sum1+v*costs[i][j]
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
               
    print("Allocation: ",bfs)
    print("\nCost by NWCR: ",sum1)
    print("\n")
    return bfs

def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: continue
                
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
            
    return us, vs

def get_ws(bfs, costs, us, vs):
    ws = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                ws.append(((i, j), us[i] + vs[j] - cost))
    
    return ws

def can_be_improved(ws):
    for p, v in ws:
        if v > 0: return True
    return False

def get_entering_variable_position(ws):
    ws_copy = ws.copy()
    ws_copy.sort(key=lambda w: w[1])
    return ws_copy[-1][0]

def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row

def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_position])

def loop_pivoting(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs

def transportation_method(supply, demand, costs, penalties = None):
    balanced_supply, balanced_demand, balanced_costs = get_balanced_tp(supply, demand, costs)
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, balanced_costs)
        ws = get_ws(bfs, balanced_costs, us, vs)
        if can_be_improved(ws):
            ev_position = get_entering_variable_position(ws)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))
        return bfs
    
    basic_variables = inner(north_west_corner(balanced_supply, balanced_demand,costs))
    solution = np.zeros((len(costs), len(costs[0])))
    for (i, j), v in basic_variables:
        solution[i][j] = v

    return solution

def get_total_cost(costs, solution):
    total_cost = 0
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            total_cost += cost * solution[i][j]
    return total_cost

costs = [
        [7,6,4,5,9],
        [8,5,6,7,8],
        [6,8,9,6,5],
        [5,7,7,8,6]
    ]
supply = [40,30,20,10]
demand = [30,30,15,20,5]
solution = transportation_method(supply, demand, costs)
print(solution)
print('\nTotal Cost after optimization: ', get_total_cost(costs, solution))

Allocation:  [((0, 0), 30), ((0, 1), 10), ((1, 1), 20), ((1, 2), 10), ((2, 2), 5), ((2, 3), 15), ((3, 3), 5), ((3, 4), 5)]

Cost by NWCR:  635


[[ 5.  0. 15. 20.  0.]
 [ 0. 30.  0.  0.  0.]
 [15.  0.  0.  0.  5.]
 [10.  0.  0.  0.  0.]]

Total Cost after optimization:  510.0


In [ ]:
# MODI Method via LCM

import numpy as np

def getBalanced(supply, demand, costs, penalties = None):
    total_supply = sum(supply)
    total_demand = sum(demand)
    if total_supply < total_demand:
        if penalties is None:
            raise Exception('Supply less than demand, penalties required')
        new_supply = supply + [total_demand - total_supply]
        new_costs = costs + [penalties]
        return new_supply, demand, new_costs
    if total_supply > total_demand:
        new_demand = demand + [total_supply - total_demand]
        new_costs = costs + [[0 for _ in demand]]
        return supply, new_demand, new_costs
    if total_demand == total_supply:
        print("\nSince total demand is equal to total supply,\nthe transportation problem is balanced")
    return supply, demand, costs

def LCM(supply, demand, costs):
    inf = np.iinfo(np.int32).max
    m=len(supply)
    n=len(demand)
    supply_copy = np.array(supply).reshape(m)
    demand_copy = np.array(demand).reshape(n)
    i = 0
    j = 0
    cost_matrix = np.array(costs).reshape(m, n)
    bfs = []
    min_cost = np.amin(cost_matrix)

    # IBFS Calculation
    while min_cost != inf:
        indices = np.where(cost_matrix == min_cost)
        i = indices[0][0]
        j = indices[1][0]
        v = min(supply_copy[i], demand_copy[j])
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i,j), v))
        if supply_copy[i] < demand_copy[j]:
            j = 0
            while j < n:
                cost_matrix[i][j] = inf
                j += 1
        elif supply_copy[i] > demand_copy[j]:
            i = 0
            while i < m:
                cost_matrix[i][j] = inf
                i += 1
        else:
            k = 0
            while k < n:
                cost_matrix[i][k] = inf
                k += 1
            k = 0
            while k < m:
                cost_matrix[k][j] = inf
                k += 1
        min_cost = np.amin(cost_matrix)

    # Output
    cost = 0
    bfs_arr = [[0 for i in range(n)] for j in range(m)] 
    for item in bfs:
        bfs_arr[item[0][0]][item[0][1]]=item[1]
    print('\nThe initial basic feasible solution is:\n',bfs_arr)
    for item in bfs:
        cost=cost+costs[item[0][0]][item[0][1]]*item[1]
    print('Total optimal solution cost is: ', cost)
    return bfs

def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: continue
                
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
    return us, vs

def get_ws(bfs, costs, us, vs):
    ws = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                ws.append(((i, j), us[i] + vs[j] - cost))
    return ws

def canBeImproved(ws):
    for p, v in ws:
        if v > 0: return True
    return False

def getEnteringVariablePosition(ws):
    ws_copy = ws.copy()
    ws_copy.sort(key=lambda w: w[1])
    return ws_copy[-1][0]

def getPossibleNextNodes(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row

def getLoop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(getPossibleNextNodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = getPossibleNextNodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    return inner([ev_position])

def loopPivoting(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
    return new_bfs

def transportationMethod(supply, demand, costs, penalties = None):
    balanced_supply, balanced_demand, balanced_costs = getBalanced(supply, demand, costs)
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, balanced_costs)
        ws = get_ws(bfs, balanced_costs, us, vs)
        if canBeImproved(ws):
            ev_position = getEnteringVariablePosition(ws)
            loop = getLoop([p for p, v in bfs], ev_position)
            return inner(loopPivoting(bfs, loop))
        return bfs
    basic_variables = inner(LCM(balanced_supply, balanced_demand,costs))
    ans = np.zeros((len(costs), len(costs[0])))
    for (i, j), v in basic_variables:
        ans[i][j] = int(v)
    return ans

def getTotalCost(costs, ans):
    total_cost = 0
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            total_cost += cost * ans[i][j]
    return total_cost

m = int(input("Enter supply count: "))
n = int(input("Enter demand count: "))
print("Enter all costs in one line:")

entries = list(map(int, input().split()))  
costs = np.array(entries).reshape(m, n)
print('\nThe cost matrix is:\n',costs)

supply = list(map(int,input("\nEnter the supply values: ").strip().split()))[:m]
demand = list(map(int,input("Enter the demand values: ").strip().split()))[:n]
ans = transportationMethod(supply, demand, costs)

print('\nThe optimal solution is:\n',ans)
print('Total optimal cost: ', getTotalCost(costs, ans))

Enter supply count: 4
Enter demand count: 5
Enter all costs in one line:
7 6 4 5 9 8 5 6 7 8 6 8 9 6 5 5 7 7 8 6

The cost matrix is:
 [[7 6 4 5 9]
 [8 5 6 7 8]
 [6 8 9 6 5]
 [5 7 7 8 6]]

Enter the supply values: 40 30 20 10
Enter the demand values: 30 30 15 20 5

Since total demand is equal to total supply,
the transportation problem is balanced

The initial basic feasible solution is:
 [[5, 0, 15, 20, 0], [0, 30, 0, 0, 0], [15, 0, 0, 0, 5], [10, 0, 0, 0, 0]]
Total optimal solution cost is:  510
